# 1. 컨텐츠 기반 추천

## 1-1. 컨텐츠 기반 모델

1) 정의 : 사용자가 이전에 구매한 상품 중에서 좋아하는 상ㅍ무들과 유사한 상품들을 추천하는 방법

2) Represented Items : 텍스트나 이미지 같은 아이템을 벡터 형태로 표현하여, 벡터들간의 유사도를 계산해 벡터1부터 N까지 자신과 유사한 벡터를 추출한다.

3) 유사도 함수
- 유클리디안 유사도 : 문서간의 유사도를 계산(1 / (유클리디안 거리 + 1e-05))
ㄴ 장점 : 계산하기 쉽다
ㄴ 단점 : p와 q의 분포가 다르거나 범위가 다른 경우에 상관성을 놓친다

- 코사인 유사도 : 문서간의 유사도를 계산
ㄴ 장점 : 벡터의 크기가 중요하지 않은 경우에 거리를 측정하기 위한 메트릭으로 사용
ㄴ 단점 : 벡터의 크기가 중요한 경우에 대해 잘 작동하지 않는다

- 피어슨 유사도 : 상관관계를 분석할 때 주로 사용한다

- 자카드 유사도 : 집합에서 얼마만큼의 결합된 부분이 있는지를 확인

- 그외 여러 유사도 함수가 있으므로 scikit-learn 등을 확인하여 여러 유사도 함수를 확인해 볼 필요가 있다
- 여러 유사도 함수를 확인한 뒤 유사도 함수 별로 가중치를 주는 등 결합하여 사용할 수 있다, 또한 각각의 매트릭스 별로 비교할 수도 있다






## 1-2. TF-IDF

1) TF-IDF : 텍스트 아이템을 벡터로 표현하는 방법
- 정의

ㄴ 단어빈도(TF) : 특정 문서 d에 특정 단어 t가 얼마나 자주 등장하는지(등장횟수)

ㄴ 역문서빈도(DF) : 전체 문서에서 특정 단어 t가 얼마나 자주 등장하는지(t가 등장한 문서의 수)

ㄴ IDF(d,t) : DF에 반비례하는 수 IDF(d, t) = log(n / 1 + df(t))

ㄴ TF와 DF를 통해 "다른 문서에서는 등장하지 않지만 특정 문서에서만 자주 등장하는 단어"를 찾아 문서 내 단어의 가중치를 계산


- 용도 : 문서의 핵심어를 추출, 문서들 사이의 유사도 계산, 검색 결과의 중요도를 정하는 작업 등에 사용

- TF(d, t) * IDF(d, t) = TF-IDF(d, t)

- 사용하는 이유?

ㄴ item이라는 컨텐츠를 벡터로 "Feature Extract" 과정을 수행

ㄴ 빈도수를 기반으로 많이 나오는 중요한 단어들을 잡아준다(counter vectorizer)

ㄴ 하지만 이는 단순 빈도만을 계산하기에 조사/관사처럼 의미없지만 문장에 많이 등장하는 단어들을 높게 칠 수 있음. 이러한 단어들에 패널티를 주어 중요한 단어를 잡아내는 기법이 TF-IDF


- 장점 : 직관적인 해석

- 단점 : 대규모 말뭉치를 다룰 때 메모리상의 문제가 발생하고, 높은 차원을 가지게 된다. 매우 sparse한 형태의 데이터..


In [14]:
docs = ['먹고 싶은 사과', # 문서1
       '먹고 싶은 바나나',# 문서2
       '길고 노란 바나나 바나나',# 문서3
       '저는 과일이 좋아요']# 문서4

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [16]:
#문장을 counter vectorizer 형태로 표현

countvect = vect.fit_transform(docs)
countvect

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [17]:
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [18]:
# 딕셔너리 형태로 각각의 단어의 컬럼 위치를 보여준다.
vect.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [19]:
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [20]:
import pandas as pd

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1','문서2','문서3','문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [21]:
# 0번 문서는 1번 문서와 유사하다는 결론을 얻을 수 있다. 따라서 0번 문서를 본 사용자에게 1번 문서를 추천할 수 있다.
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [22]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)

In [23]:
tfvect_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfvect_df.index = ['문서1','문서2','문서3','문서4']
tfvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfvect_df, tfvect_df)

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

### 영화데이터 실습

In [25]:
#https://www.kaggle.com/rounakbanik/the-movies-dataset?select=movies_metadata.csv

data = pd.read_csv('C:/Users/sbpark/movies_metadata.csv', low_memory = False)
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [26]:
data['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [27]:
#전처리
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [28]:
data = data[data['overview'].notnull()].reset_index(drop = True)
data.shape

(44512, 24)

In [29]:
data = data.loc[0:20000].reset_index(drop = True)

In [32]:
#불용어 : 유의미하지 않은 단어 토근 제거

tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20001, 47665)


In [33]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [34]:
cosine_matrix.shape

(20001, 20001)

In [36]:
import numpy as np
np.round(cosine_matrix, 4)

array([[1.    , 0.0158, 0.    , ..., 0.0083, 0.0172, 0.    ],
       [0.0158, 1.    , 0.0492, ..., 0.0057, 0.008 , 0.    ],
       [0.    , 0.0492, 1.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.0083, 0.0057, 0.    , ..., 1.    , 0.0144, 0.    ],
       [0.0172, 0.008 , 0.    , ..., 0.0144, 1.    , 0.0183],
       [0.    , 0.    , 0.    , ..., 0.    , 0.0183, 1.    ]])

In [40]:
#movie title과 id를 매핑할 dictionary 생성

movie2id = {}
for i, c in enumerate(data['title']) : movie2id[i] = c
id2movie = {}

for i,c in movie2id.items() : id2movie[c] = i

In [44]:
#toystory의 id 추출

idx = id2movie['Toy Story']
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
sim_scores

[(1, 0.01577499623706559),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.041138683296865486),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0099121496903153),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.01978034381431984),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.006321775635368981),
 (42, 0.0),
 (43, 0.0),
 (44, 0.009292791126667362),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.013838678611953216),
 (49, 0.009852367947354567),
 (50, 0.010928162091485132),
 (51, 0.0),
 (52, 0.0),
 (53, 0.02000467244181858),
 (54, 0.0),
 (55, 0.025263801435198463),
 (56, 0.02072192444202655),
 (57, 0.0),
 (58, 0.03420184247473588),
 (59, 0.0),
 (60, 0.0),
 (61, 0.00860353886947865),
 (62, 0.0),
 (63, 0.01019819462957017),
 (64, 0

In [45]:
sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
sim_scores[0:10] # 상위 10개의 인덱스와 유사도 추출

[(15282, 0.5262275451171008),
 (2979, 0.463276799830381),
 (10271, 0.2797390476075632),
 (8303, 0.20078538664316947),
 (1058, 0.18287334034120212),
 (11367, 0.15712074193481165),
 (1916, 0.15288512626542436),
 (3039, 0.1433450408051554),
 (483, 0.13765225108436677),
 (11573, 0.1337032693869044)]

In [47]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Toy Story 3', 0.5262275451171008),
 ('Toy Story 2', 0.463276799830381),
 ('The 40 Year Old Virgin', 0.2797390476075632),
 ('The Champ', 0.20078538664316947),
 ('Rebel Without a Cause', 0.18287334034120212),
 ('For Your Consideration', 0.15712074193481165),
 ('Condorman', 0.15288512626542436),
 ('Man on the Moon', 0.1433450408051554),
 ('Malice', 0.13765225108436677),
 ('Factory Girl', 0.1337032693869044)]